<a href="https://colab.research.google.com/github/alevinas/minecraftImagesRNA/blob/main/minecraftImagesRecognize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load libraries

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimag
import zipfile as zipFile
import os
import re
import cv2
from PIL import Image
from random import randint

In [ ]:
# Own dataset with minecraft mobs images from Github
!wget --no-check-certificate \
    "https://github.com/alevinas/minecraftImagesRNA/archive/refs/heads/main.zip" \
    -O "/content/minecraftImagesRNA.zip"


zip_ref = zipFile.ZipFile('/content/minecraftImagesRNA.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/content') #Extracts the files into the /content folder
zip_ref.close()

!rm minecraftImagesRNA.zip # Erase the zip

In [ ]:
# Auxiliar function to better prints
def boldText(text):
  return "\033[1m" + text + "\033[0m"

pathDirDataset = '/content/minecraftImagesRNA-main/dataset'
pathTrain = '/train'
pathTest = '/test'

In [ ]:
#@markdown ### Defina los parámetros de las imágenes:
imagen_largo_ancho =  64#@param {type:"integer"}
imagen_color = True #@param {type:"boolean"}

if imagen_largo_ancho <= 15: # Minimum size
  imagen_largo_ancho = 15
image_shape = (imagen_largo_ancho, imagen_largo_ancho, 3 if imagen_color == True else 1) # 3 color RGB y 1 BN
tam_input = image_shape[0] * image_shape[1] * image_shape[2] # Image size


#@markdown ### Defina los parámetros de la RNA:
cantEpochs = 200 #@param {type:"integer"}

print("Configuración para las imágenes")
print("Tamaño de la imagen: ", image_shape)
print("=================================")
print("Configuración para la RNA")
print("Cantidad de epocas definidas:", cantEpochs)

In [ ]:
# Processing image

def processImage(img):
  if(image_shape[2]==1):
    imageType = 'L' # Black and White
  else:
    imageType = 'RGB'
  img = img.convert(imageType)
  img = img.resize((image_shape[0], image_shape[1]), Image.ANTIALIAS) # With anti-aliansing
  return img, imageType

# I define the function to load the images

def loadImages(dirPath):
  images = []
  mobs = []

  allDirMobs = os.listdir(dirPath) ## List all folders
  for dir in allDirMobs:
    if not dir.startswith('.'):
      pathAux = dirPath + os.sep + dir
      allImagesMob = os.listdir(pathAux) ## List all images by folder
      for img in allImagesMob:
        if re.search('\.(jpg|jpeg|png|bmp)$', img):
          imgPath = Image.open(pathAux + os.sep + img)
          img, imageType = processImage(imgPath)
          img = np.array(img) # I want a pixels array
          images.append(img)
          mobs.append(dir) # mobs.append(dir.capitalize()) --> TODO
  return images, mobs, imageType

images_train, mobs_train, imageType_train = loadImages(pathDirDataset + pathTrain)

print(boldText("Datos cargados sobre entrenamiento:"))
print("- Mobs cargadas: ", len(mobs_train))
print("- Imágenes cargadas: ", len(images_train))
print("- Ejemplo:")
display(Image.fromarray(images_train[2]))

images_test, mobs_test, imageType_test = loadImages(pathDirDataset + pathTest)

print(boldText("Datos cargados sobre test:"))
print("- Mobs cargadas: ", len(mobs_test))
print("- Imágenes cargadas: ", len(images_test))
print("- Ejemplo:")
display(Image.fromarray(images_train[3]))


In [ ]:
# Generate labels by mob to supervised algorithm of the RNA
# Normalization 
def prepareImgList(imgList):
  aux = np.array(imgList).astype('float32') / 255
  aux = aux.reshape((len(aux),tam_input))
  return np.array(aux)

# With One Hot Encoded Data
def prepareMobList(mobsList, dictionary=None):
  if dictionary==None:
    auxDict = list(set(mobsList))
    dictionary = dict(zip(auxDict, range(len(auxDict))))
  # Mapping
  y = []
  for mob in mobsList:
      y.append(dictionary[mob])
  # To transform a numeric array to binary array
  oneHot_y = tf.keras.utils.to_categorical(y)

  return np.array(y), np.array(oneHot_y), dictionary

x_train = prepareImgList(images_train)
x_test = prepareImgList(images_test)

y_train, oneHot_y_train, dictionary = prepareMobList(mobs_train)
y_test, oneHot_y_test, _  = prepareMobList(mobs_test)

# Auxiliar dictionary
mobs = [x for x,y in dictionary.items()]

# Plot image with different size
def plotImage(img):
  if image_shape[2]==1:
    plt.imshow((img*255).reshape(image_shape[0], image_shape[1]).astype(np.uint8))
    plt.gray()
  else:
    plt.imshow((img*255).reshape(image_shape).astype(np.uint8))
  plt.axis("off") 

print(boldText("Entrenamiento:"))
print("(Cantidad de imágenes, Cantidad píxeles por imagen)", x_train.shape, end="\n\n")

print(boldText("Prueba:"))
print("(Cantidad de imágenes, Cantidad píxeles por imagen)", x_test.shape, end="\n\n")

print(boldText("General:"))
print("Diccionario:", dictionary)
print("Mobs:", mobs, end="\n\n")

print(boldText("Ejemplo:"))
print("Imagen de:", mobs[y_train[3]], "( Dictionary ID:", y_train[3], "|| One Hot Code", oneHot_y_train[3], ")")
plotImage(x_train[3])

In [ ]:
# Construct the RNA

